# Model Context Protocol (MCP) - A Guide for Developers

## Introduction
The **Model Context Protocol (MCP)** is an open standard that enables AI models (like Claude, GPT-4) to safely interact with local and remote resources. Think of it as a "USB-C for AI apps"—a standard way to connect LLMs to data sources and tools without building custom integrations for every new model or application.

### Why learn MCP?
- **Standardization**: Build a tool once, works with any MCP-compliant client (Claude Desktop, IDEs like Cursor/Windsurf, etc.).
- **Security**: Explicit permissions model.
- **Flexibility**: Expose local files, databases, or API endpoints as "Resources" or "Tools".

## Core Concepts
1. **MCP Server**: The program that exposes capabilities (tools, resources, prompts).
2. **MCP Client**: The application that consumes these capabilities (e.g., Claude Desktop, specific IDE extensions).
3. **Resources**: Data that can be read by the LLM (like files, database rows).
4. **Tools**: Functions that the LLM can execute (like `calculate_sum`, `fetch_weather`).
5. **Prompts**: Pre-defined templates for interaction.

---

## 1. Setup
We will need the `mcp` python package.

In [ ]:
%pip install mcp

## 2. Creating a Simple MCP Server

We will create a server using the `FastMCP` high-level interface (if available in the library version) or the standard low-level server builders. For this tutorial, we will focus on the conceptual structure which is often implemented as a standalone script. 

Below is an example of how to define a server that provides:
1. A **Tool** to calculate the rate of return.
2. A **Resource** to read a dynamic "note".

**Note**: In a real scenario, you run this as a standalone python script (e.g., `python server.py`) and point your MCP Client to it.

In [ ]:
from mcp.server.fastmcp import FastMCP

# Initialize the server
mcp = FastMCP("FinancialHelper")

# --- 1. Define a Tool ---
# Tools are functions the LLM can call to perform actions or get calculated data.

@mcp.tool()
def calculate_compound_interest(principal: float, rate: float, time: int) -> str:
    """
    Calculates compound interest.
    
    Args:
        principal: The initial amount of money.
        rate: The annual interest rate (in percent, e.g., 5 for 5%).
        time: The time the money is invested for, in years.
    """
    amount = principal * (pow((1 + rate / 100), time))
    return f"The amount after {time} years will be {amount:.2f}"

# --- 2. Define a Resource ---
# Resources are passive data sources (like files) the LLM can read.
# The URI scheme "financial://" is custom to our server.

@mcp.resource("financial://notes")
def get_daily_market_notes() -> str:
    """Returns today's market summary notes."""
    return """
    # Market Summary
    - Tech stocks are up 2% today.
    - Bond yields are stable.
    - Recommended action: Hold.
    """

if __name__ == "__main__":
    # This runs the server over stdio, which is what MCP clients expect by default
    mcp.run()

## 3. How to Connect this to a Client (e.g., Claude Desktop)

To use the code above:

1. Save the code into a file named `my_mcp_server.py`.
2. Locate your Claude Desktop configuration file:
   - **Mac**: `~/Library/Application Support/Claude/claude_desktop_config.json`
   - **Windows**: `%APPDATA%\Claude\claude_desktop_config.json`
3. Add the following entry:

```json
{
  "mcpServers": {
    "financial-helper": {
      "command": "python",
      "args": ["/absolute/path/to/my_mcp_server.py"]
    }
  }
}
```

4. Restart Claude Desktop. You will see a plug icon, and you can now ask Claude: "Calculate compound interest for 1000 dollars at 5% for 10 years."

## 4. Advanced: Using Prompts

MCP also allows you to define **Prompts**. These are reusable templates that become slash commands or menu items in the client.


In [ ]:
@mcp.prompt()
def investment_advice(risk_level: str) -> str:
    """Generate investment advice based on risk level."""
    return f"Please act as a financial advisor. The user has a {risk_level} risk tolerance. Suggest a portfolio allocation."


## 5. Inspecting MCP Traffic

Since MCP runs over Stdio (Standard Input/Output) by default, debugging can be tricky. You can use the **MCP Inspector** tool provided by the community or simply log to a file (do not print to stdout, as that corrupts the protocol JSON-RPC messages).

## Example: SQLite Explorer MCP Server

Here is a more complex example idea: A server that connects to a local SQLite database and lets the LLM run queries.

**Tool**: `run_query(sql_query)`
**Resource**: `sqlite://schema` (returns the database schema)

This allows the LLM to understand your local data structure and query it to answer questions.

## 6. Real-World Examples of MCP Servers

The community is building a rich ecosystem of servers. Here are some popular ones you might encounter or want to build:

- **Filesystem Server**: Allows the LLM to read/write files in a specific directory (safe sandbox).
- **PostgreSQL / SQLite Server**: Gives the LLM read-only (or read-write) access to database schemas and content to answer "business intelligence" questions.
- **Git / GitHub Server**: Allows the LLM to search repositories, read PRs, or check diffs.
- **Web Search Server**: Wraps APIs like Brave Search or Google Search to give the LLM internet access.
- **Slack / Discord Server**: Allows the LLM to read recent messages or post summaries.

Find more at the official [MCP Registry or GitHub topics](https://github.com/topics/mcp).

## Summary

- **MCP** standardizes LLM tools.
- Use `FastMCP` for quick Python servers.
- Run over `stdio`.
- Configure Client via JSON config.